In [1]:
import numpy as np

In [19]:
num_companies = 4  # кол-во компаний
capital = 700  # в тыс. руб.
capital_delta = 100
profit_of_investment = np.array([[0, 28, 45, 65, 78, 90, 102, 113],
                                 [0, 25, 41, 55, 65, 75, 80, 85],
                                 [0, 15, 25, 40, 50, 62, 73, 82],
                                 [0, 20, 33, 42, 48, 53, 56, 58]])
# profit_of_investment = np.array([[0, 7, 13, 20, 33, 35],
#                                  [0, 8, 12, 21, 28, 40],
#                                  [0, 10, 11, 19, 30, 34],
#                                  [0, 6, 9, 24, 30, 36]])

In [110]:
# итоговая функция

def foo(num_companies, capital, A: np.array):
    """
    :param num_companies: количество предприятий
    :param capital: капитал
    :param profit_of_investment: двумерный массив
    :return:
    """
    F = {}  # F[(k, i)] means F_k(i)
    for k in range(num_companies - 1, -1, -1):
        X = [i for i in range(0, capital + capital_delta, capital_delta)]  # возможные варианты x
        for x in X:
            if k == num_companies - 1:
                F[(k, x)] = A[k][X.index(x)]
            else:
                U = [i for i in range(0, x + capital_delta, capital_delta)]  # возможные варианты u
                F[(k, x)] = np.max([A[k][X.index(x)] + F[(k + 1, x - u)] for u in U]) # создаем массив из возможных F и находим в нем максимум
            print("F[({0}, {1})] = ".format(k, x), F[(k, x)])

    print("Answer = ", F[(0, capital)])

    for i in range(1):
        # print("Variant 1:")
        pass
        for j in range(1):
            # print("It should be spent (*amount1*) on company (*number1*)")
            # print("Total profit is (*target value*))")
            pass

In [111]:
foo(num_companies, capital, profit_of_investment)

F[(3, 0)] =  0
F[(3, 100)] =  20
F[(3, 200)] =  33
F[(3, 300)] =  42
F[(3, 400)] =  48
F[(3, 500)] =  53
F[(3, 600)] =  56
F[(3, 700)] =  58
F[(2, 0)] =  0
F[(2, 100)] =  35
F[(2, 200)] =  58
F[(2, 300)] =  82
F[(2, 400)] =  98
F[(2, 500)] =  115
F[(2, 600)] =  129
F[(2, 700)] =  140
F[(1, 0)] =  0
F[(1, 100)] =  60
F[(1, 200)] =  99
F[(1, 300)] =  137
F[(1, 400)] =  163
F[(1, 500)] =  190
F[(1, 600)] =  209
F[(1, 700)] =  225
F[(0, 0)] =  0
F[(0, 100)] =  88
F[(0, 200)] =  144
F[(0, 300)] =  202
F[(0, 400)] =  241
F[(0, 500)] =  280
F[(0, 600)] =  311
F[(0, 700)] =  338
Answer =  338
